In [19]:
import pandas as pd
import pyxlsb

In [ ]:
estoque_dtype = {
    'Código Produto':int,
    'Código Estabelecimento':int,
    'Total': 'int64',
    'Data':object
}


estoque = pd.read_excel("Estoque_Por_Dia.xlsx",
usecols=['Código Produto','Código Estabelecimento','Total','Data'],
parse_dates=['Data'],
dtype=estoque_dtype)


estoque.rename(columns={'Total':'Estoque'},inplace=True)

In [ ]:
#estoque.head(10)

In [ ]:
Vendas_dtype = {
    'Código Produto':int,
    'Código Estabelecimento':int,
    'Quantidade Faturada': 'int64',
    'Fat Bruto':float,
    'Dia do Mês':object
}



Vendas = pd.read_excel('Cubo de Vendas.xlsx',skiprows=9,
usecols=['Código Produto','Código Estabelecimento','Quantidade Faturada','Fat Bruto','Dia do Mês'],
parse_dates=['Dia do Mês'],
dtype=Vendas_dtype)

Vendas.rename(columns={'Dia do Mês':'Data'})


KeyboardInterrupt: 

In [ ]:
Vendas.head(10)

,Código Estabelecimento,Código Produto,Quantidade Faturada,Fat Bruto
0,811,4111140179,104,143906.96
1,811,4112856753,656,78080.84
2,811,4111110330,83,65555.17
3,811,4111172037,31,51523.69
4,811,4111110352,25,49099.75
5,811,4111175639,31,39704.69
6,811,4111110294,5,34689.95
7,811,4111171049,655,31998.02
8,811,4111176968,12,29599.88
9,811,4111110357,16,21404.86
